In [1]:
import matplotlib.pyplot as plt
import numpy as np
from casadi import *
from NMPC_quat import NMPC_quat
%load_ext autoreload

In [2]:
%autoreload
Q = np.diag([120,
             100,
             100,
             1,
             1,
             1,
             1,
             7e-1,
             1.0,
             4.0,
             1e-5,
             1e-5,
             1e-5])

# Q_position = np.diag([120,
#              100,
#              100])



R = np.diag([1, 1, 1, 1])* 0.06


In [3]:
x_init = np.array([0,0,0.2])
x_des = np.array([0.3,0.5,0.21])

N = 10 # horizon length
T = 0.1 # time step
Tf = 30 # simulation time
method = "DMS"
# method = "DC"; degree = 2
store_prediction = True
nlp_opts_dms = { "ipopt.tol": 1e-3, "ipopt.max_iter": 100}


In [4]:
# initialise NMPC instance
dms_mpc = NMPC_quat(Q, R, N, T, Tf, method, store_prediction=store_prediction, nlpopts_dms=nlp_opts_dms)

dms_mpc.set_values(x_init, x_des)


In [5]:
print("Starting state : ",dms_mpc.x_init)
print("Desired state : ",dms_mpc.x_des)

Starting state :  [0.  0.  0.2 1.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
Desired state :  [0.3  0.5  0.21 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]


In [6]:
print("Solver : ",dms_mpc.solver)

Solver :  solver:(x0[183],p[30],lbx[183],ubx[183],lbg[156],ubg[156],lam_x0[183],lam_g0[156])->(x[183],f,g[156],lam_x[183],lam_g[156],lam_p[30]) IpoptInterface


In [7]:
print("method : ",dms_mpc.method)

method :  DMS


In [8]:
print("Size of optimisation variable vector : ",len(dms_mpc.w0))

Size of optimisation variable vector :  183


In [9]:
dms_mpc.solve_open_loop()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1676
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1591

Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 1
Number of 

CasADi - 2023-02-08 17:03:26 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 170, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2023-02-08 17:03:26 WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 170, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2023-02-08 17:03:26 WARNING("solver:nlp_grad failed: NaN detected for output grad_gamma_p, at (row 17, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2023-02-08 17:03:26 WARNING("Failed to calculate multipliers") [.../casadi/core/nlpsol.cpp:621]


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(dms_mpc.X_mpc[:,0],dms_mpc.X_mpc[:,1],dms_mpc.X_mpc[:,2], 'o')
ax.scatter(dms_mpc.x_init[0],dms_mpc.x_init[1],dms_mpc.x_init[2],c='r')
ax.scatter(dms_mpc.x_des[0],dms_mpc.x_des[1],dms_mpc.x_des[2],c='g')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()


In [ ]:
fig = plt.figure()
plt.stairs(dms_mpc.U_mpc[:,0],label='u1')
plt.stairs(dms_mpc.U_mpc[:,1],label='u2')
plt.stairs(dms_mpc.U_mpc[:,2],label='u3')
plt.stairs(dms_mpc.U_mpc[:,3],label='u4')
plt.legend()
plt.show()




In [ ]:
dms_mpc.thrust_list

In [ ]:
dms_mpc.L

In [ ]:
#print phi_list, theta list and psi_list as an array with 3 columnns and each row contains the values of phi, theta and psi at a time step
fig = plt.figure()
plt.stairs(dms_mpc.phi_list,label='phi')
plt.stairs(dms_mpc.theta_list,label='theta')
plt.stairs(dms_mpc.psi_list,label='psi')
plt.legend()
plt.show()





In [ ]:
import matplotlib.pyplot as plt
%load_ext autoreload
import numpy as np
from casadi import *
from NMPC_quat import NMPC_quat
# %load_ext autoreload
# solve the closed loop system
%autoreload
Q = np.diag([120,
             100,
             100,
             1,
             1,
             1,
             1,
             7e-1,
             1.0,
             4.0,
             1e-2,
             1e-2,
             1e-2])

# Q_position = np.diag([120,
#              100,
#              100])



R = np.diag([1, 1, 1, 1])*0.06
x_init = np.array([0,0,0.5])
x_des = np.array([0.1,0.1,0.5])
deviation_threshold = 0.1
N =10 # horizon length
T = 0.5 # time step
Tf = 10# simulation time
method = "DMS"
# method = "DC"; degree = 2
store_prediction = True
nlp_opts_dms = { "ipopt.tol": 1e-3, "ipopt.max_iter": 30, "ipopt.print_level": 0, "print_time": 0}

# initialise NMPC instance
dms_mpc = NMPC_quat(Q, R, N, T, Tf, method, store_prediction=store_prediction, nlpopts_dms=nlp_opts_dms)

dms_mpc.set_values(x_init, x_des)
print("Starting state : ",dms_mpc.initial_guess_state)
print(dms_mpc.N_sim)
stable_counter = 0
for i in range(dms_mpc.N_sim):
    # print("Iteration : ",i)
    dms_mpc.solve_for_next_state()
    dms_mpc.extract_next_state(i)
    # print("current pose =", dms_mpc.X_opt_current[7:10])
    # print("Current quaternion = ", dms_mpc.X_opt_current[3:7])
    # print("Control = ", dms_mpc.U_opt_current)
    if dms_mpc.deviation_from_reference < deviation_threshold:
        print("stability counter = ", stable_counter)
        stable_counter += 1
        if stable_counter > 10:
            print("System is stable")
            print("System converged in " , i, " steps")
            print("Final state : ",dms_mpc.X_opt_current)
            break
    if i >  dms_mpc.N_sim - 1:
        print("System is unstable")
        break



In [ ]:
# plot the 3d trajectory
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x_des[0], x_des[1], x_des[2], 'ro', label='Desired position', markersize=10)
ax.plot(dms_mpc.X_mpc[i,0], dms_mpc.X_mpc[i,1], dms_mpc.X_mpc[i,2], 'go', label='Final position')
ax.plot(x_init[0], x_init[1], x_init[2], 'bo', label='Initial position')
ax.plot(dms_mpc.X_mpc[:i,0], dms_mpc.X_mpc[:i,1], dms_mpc.X_mpc[:i,2],'yo')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.legend()
plt.show()



In [ ]:
#plot the controls
plt.figure()
plt.stairs(dms_mpc.U_mpc[:i,0], label="u1")
plt.stairs(dms_mpc.U_mpc[:i,1], label="u2")
plt.stairs(dms_mpc.U_mpc[:i,2], label="u3")
plt.stairs(dms_mpc.U_mpc[:i,3], label="u4")
plt.legend()
plt.savefig("control_for_y_mvmt.png")
plt.show()


In [ ]:
dms_mpc.control_list

In [ ]:
print(dms_mpc.X_mpc[:i,3:7])

In [ ]:
import matplotlib.pyplot as plt
%load_ext autoreload
import numpy as np
from casadi import *
from NMPC_quat import NMPC_quat
# %load_ext autoreload
# solve the closed loop system
%autoreload
Q = np.diag([120,
             100,
             100,
             1,
             1,
             1,
             1,
             1,
             1.0,
             10.0,
             1e-2,
             1e-2,
             1e-2])

# Q_position = np.diag([120,
#              100,
#              100])



R = np.diag([1, 1, 1, 1])*10
x_init = np.array([0,0,0.2])
x_des = np.array([0.2,0.2,0.2])
deviation_threshold = 0.05
N =20# horizon length
T = 0.05 # time step
Tf = 5# simulation time
# method = "DMS"
method = "DC"; degree = 2
store_prediction = True
nlp_opts_dc = { "ipopt.tol": 1e-3, "ipopt.max_iter": 50, "ipopt.print_level": 0, "print_time": 0}

# initialise NMPC instance
dc_mpc = NMPC_quat(Q, R, N, T, Tf, method, store_prediction=store_prediction, nlpopts_dc = nlp_opts_dc, degree=degree)

dc_mpc.set_values(x_init, x_des)
print("Starting state : ",dc_mpc.initial_guess_state)
print(dc_mpc.N_sim)
print(dc_mpc.deviation_from_reference)
stable_counter = 0
for i in range(dc_mpc.N_sim):
    print("Iteration : ",i)
    dc_mpc.solve_for_next_state()
    dc_mpc.extract_next_state(i)
    print("current pose =", dc_mpc.X_opt_current)
    # print("Current quaternion = ", dc_mpc.X_opt_current[3:7])
    # print("Control = ", dc_mpc.U_opt_current)
    print(dc_mpc.deviation_from_reference)
    if dc_mpc.deviation_from_reference < deviation_threshold:
        print("stability counter = ", stable_counter)
        stable_counter += 1
        if stable_counter > 50:
            print("System is stable")
            print("System converged in " , i, " steps")
            print("Final state : ",dc_mpc.X_opt_current)
            break
    if i >  dc_mpc.N_sim - 1:
        print("System is unstable")
        break



In [ ]:
# plot the 3d trajectory
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x_des[0], x_des[1], x_des[2], 'ro', label='Desired position', markersize=10)
ax.plot(dc_mpc.X_mpc[i,0], dc_mpc.X_mpc[i,1], dc_mpc.X_mpc[i,2], 'go', label='Final position')
ax.plot(dc_mpc.X_mpc[0,0], dc_mpc.X_mpc[0,1], dc_mpc.X_mpc[0,2], 'bo', label='Initial position')
ax.plot(dc_mpc.X_mpc[:i,0], dc_mpc.X_mpc[:i,1], dc_mpc.X_mpc[:i,2],'y')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.legend()
plt.show()



In [ ]:
#plot the controls
plt.figure()
plt.stairs(dc_mpc.U_mpc[:i,0], label="u1")
plt.stairs(dc_mpc.U_mpc[:i,1], label="u2")
plt.stairs(dc_mpc.U_mpc[:i,2], label="u3")
plt.stairs(dc_mpc.U_mpc[:i,3], label="u4")
plt.legend()
plt.show()


In [ ]:
#plot thrust values
plt.figure()
plt.stairs(dc_mpc.thrust_list)

In [ ]:
plt.figure()
plt.plot(dc_mpc.X_mpc[:i,0], label="x")
plt.plot(dc_mpc.X_mpc[:i,1], label="y")
plt.plot(dc_mpc.X_mpc[:i,2], label="z")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(dc_mpc.phi_list, label="yaw")
plt.plot(dc_mpc.theta_list, label="pitch")
plt.plot(dc_mpc.psi_list, label="roll")
plt.legend()
plt.show()

In [ ]:
dc_mpc.control_list

In [ ]:
roll = [i[0] for i in dc_mpc.control_list]
pitch = [i[1] for i in dc_mpc.control_list]
yaw_rate = [i[2] for i in dc_mpc.control_list]
thrust = [i[3] for i in dc_mpc.control_list]

plt.figure()
plt.subplot(4,1,1)
plt.plot(roll)
plt.subplot(4,1,2)
plt.plot(pitch)
plt.subplot(4,1,3)
plt.plot(yaw_rate)
plt.subplot(4,1,4)
plt.plot(thrust)

plt.show()


